In [ ]:
# %%capture
!pip install unsloth
!pip install tqdm
!pip install datasets

!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
from unsloth import FastLanguageModel
import torch
from datasets import Dataset
import pandas as pd
max_seq_length = 1024 
dtype = None 
load_in_4bit = True 

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    
)
tokenizer.max_length = max_seq_length  
tokenizer.truncation = True 

# if False:
# from unsloth import FastLanguageModel
# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "YuvrajSingh9886/Llama3.1_8b_Instruct_lora_model_Lampard", 
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     load_in_4bit = load_in_4bit,
#     token="..."
# )
FastLanguageModel.for_inference(model) 




In [ ]:
# FastLanguageModel.for_inference(model)

In [ ]:
# import ast
# import pandas as pd
# df = pd.read_excel('/kaggle/working/Frank Lampard Ghost Goal Labels with Llama3.1_8b_Instruct using Alpaca Prompt Fine Tuned.xlsx')

# for i in range(df.shape[0]):
#     print(df.at[i, 'Llama3.1 Not Fine Tuned output'])
#     temp = ast.literal_eval(df.at[i, 'Llama3.1 Not Fine Tuned output'])
#     df.at[i, 'Llama3.1 Not Fine Tuned output'] = temp

In [ ]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report, ConfusionMatrixDisplay, confusion_matrix


def extract(text):
    matches = re.findall(r'<\|start_header_id\|>assistant<\|end_header_id\|>\n\n(\w+)<\|eot_id\|>', text)
    return matches

    
def extract_finetuned(text):
  match = re.search(r'### Response:\s*(\w+)', text)
  label = match.group(1) if match else None

    
  return label
    
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""



system = ("You are provided with an input. You are required to perform stance detection "
              "on the input with output as one of the following labels - Favor, Against, "
              "Irrelevant, Neutral. The labels are self-explanatory. Remember to only use the labels provided and do not mispell its name. Only output the stance detected label and nothing else.")


def get_answers(model, df, comments, label, labels):


  count = 0
  for i in tqdm(range(df.shape[0])):

    tries=10
      
    while(tries):
          try:
            
            input_text = comments[i]
            output_text = labels[i]
    
            text = alpaca_prompt.format(system, input_text, "") + tokenizer.eos_token

            tokenized_text = tokenizer(text, return_tensors = 'pt').to('cuda:0')
            output = model.generate(**tokenized_text, temperature = 0.1, max_new_tokens = 10)
            result= tokenizer.decode(output[0])
            
            answer = extract(result)
      
            df.at[i, label] = answer[0]
            
   
            break
              
          except:
            tries -= 1
    count += 1

    if(count%200 == 0):
        print("Saving...")
        df.to_excel('/kaggle/working/Frank Lampard Ghost Goal Labels with Llama3.1_8b_Instruct using Alpaca Prompt Fine Tuned.xlsx')
  return df
        

def display_counts(df, counts):

    categories = ['Favor', 'Neutral', 'Irrelavent', 'Against']
  
    
    plt.bar(categories, counts)
    
    plt.title('Counts of Different Categories')
    plt.xlabel('Categories')
    plt.ylabel('Counts')
    
    plt.show()

count = []
def evaluate(model, label):
    df = pd.read_excel('/kaggle/input/lampard-dataset/Frank Lampard Ghost Goal Labels with Llama3.1_8b_Instruct using Alpaca Prompt Fine Tuned (6).xlsx')

    labels = df['Label']
    comments = df['Comments']
    
    df[label] = np.nan
    llm_df = get_answers(model, df, comments, label, labels)
    
    return llm_df


def metric_display(df):



    favor = 0
    neutral = 0
    irrelavent = 0
    against = 0
    
    for i in range(df.shape[0]):
        if(df.at[i, 'Comments'] == 'Favor'):
            favor += 1
        elif(df.at[i, 'Comments'] == 'Against'):
            neutral += 1
        elif(df.at[i, 'Comments'] == 'Neutral'):
            against += 1
        elif(df.at[i, 'Comments'] == 'Irrelevant'):
            irrelavent += 1
    counts = [favor, neutral, irrelavent, against]


    df['labels_to_numbers_original'] = None

    for i in range(df.shape[0]):
        if(df.at[i, 'Label']) == 'Favor':
            df.at[i, 'labels_to_numbers_original'] = 0
        elif(df.at[i, 'Label']) == 'Against':
            df.at[i, 'labels_to_numbers_original'] = 1
        elif(df.at[i, 'Label']) == 'Neutral':
            df.at[i, 'labels_to_numbers_original'] = 2
        elif(df.at[i, 'Label']) == 'Irrelevant':
            df.at[i, 'labels_to_numbers_original'] = 3
        else:
            df.at[i, 'labels_to_numbers_original'] = np.nan

    
    df['labels_to_numbers (Fine Tuned)'] = None
    df['labels_to_numbers (Not Fine Tuned)'] = None
    
    for i in range(df.shape[0]):
        if(df.at[i,'Llama3.1 Not Fine Tuned output']) == 'Favor':
            df.at[i, 'labels_to_numbers (Not Fine Tuned)'] = 0
        elif(df.at[i,'Llama3.1 Not Fine Tuned output']) == 'Against':
            df.at[i, 'labels_to_numbers (Not Fine Tuned)'] = 1
        elif(df.at[i,'Llama3.1 Not Fine Tuned output']) == 'Neutral':
            df.at[i, 'labels_to_numbers (Not Fine Tuned)'] = 2
        elif(df.at[i,'Llama3.1 Not Fine Tuned output']) == 'Irrelevant':
            df.at[i, 'labels_to_numbers (Not Fine Tuned)'] = 3
            
    for i in range(df.shape[0]):
        if(df.at[i, 'Llama3.1 Fine Tuned output']) == 'Favor':
            df.at[i, 'labels_to_numbers (Fine Tuned)'] = 0
        elif(df.at[i, 'Llama3.1 Fine Tuned output']) == 'Against':
            df.at[i, 'labels_to_numbers (Fine Tuned)'] = 1
        elif(df.at[i, 'Llama3.1 Fine Tuned output']) == 'Neutral':
            df.at[i, 'labels_to_numbers (Fine Tuned)'] = 2
        elif(df.at[i, 'Llama3.1 Fine Tuned output']) == 'Irrelevant':
            df.at[i, 'labels_to_numbers (Fine Tuned)'] = 3
    df.dropna(inplace=True)

    

    true = df['labels_to_numbers_original'].astype('int').to_numpy()
    pred = df['labels_to_numbers (Not Fine Tuned)'].astype('int').to_numpy()

    acc = accuracy_score(true, pred)
    target_names = ['favor', 'irrelavant', 'neutral', 'against']
    print(classification_report(true, pred, target_names=target_names))
    print("Accuracy without fine tuning: ", acc)
    
    cm = confusion_matrix(true, pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                  display_labels=['favor', 'irrelavant', 'neutral', 'against'])
    disp.plot()
    plt.show()
metric_display(df)
# df = evaluate(model, 'Llama3.1 Not Fine Tuned output')
# df.to_excel('/kaggle/working/Frank Lampard Ghost Goal Labels with Llama3.1_8b_Instruct using Alpaca Prompt Fine Tuned.xlsx')

In [ ]:
def metric_display(df):


    favor = 0
    neutral = 0
    irrelavent = 0
    against = 0
    
    for i in range(df.shape[0]):
        if(df.at[i, 'Comments'] == 'Favor'):
            favor += 1
        elif(df.at[i, 'Comments'] == 'Against'):
            neutral += 1
        elif(df.at[i, 'Comments'] == 'Neutral'):
            against += 1
        elif(df.at[i, 'Comments'] == 'Irrelevant'):
            irrelavent += 1
    counts = [favor, neutral, irrelavent, against]


    # df1 = df.copy()
    df['labels_to_numbers_original'] = None
    # print(df1)
    for i in range(df.shape[0]):
        if(df.at[i, 'Label']) == 'Favor':
            df.at[i, 'labels_to_numbers_original'] = 0
        elif(df.at[i, 'Label']) == 'Against':
            df.at[i, 'labels_to_numbers_original'] = 1
        elif(df.at[i, 'Label']) == 'Neutral':
            df.at[i, 'labels_to_numbers_original'] = 2
        elif(df.at[i, 'Label']) == 'Irrelevant':
            df.at[i, 'labels_to_numbers_original'] = 3
        else:
            df.at[i, 'labels_to_numbers_original'] = np.nan

    
    df['labels_to_numbers (Fine Tuned)'] = None
    df['labels_to_numbers (Not Fine Tuned)'] = None
    
    for i in range(df.shape[0]):
        if(df.at[i,'Llama3.1 Not Fine Tuned output']) == 'Favor':
            df.at[i, 'labels_to_numbers (Not Fine Tuned)'] = 0
        elif(df.at[i,'Llama3.1 Not Fine Tuned output']) == 'Against':
            df.at[i, 'labels_to_numbers (Not Fine Tuned)'] = 1
        elif(df.at[i,'Llama3.1 Not Fine Tuned output']) == 'Neutral':
            df.at[i, 'labels_to_numbers (Not Fine Tuned)'] = 2
        elif(df.at[i,'Llama3.1 Not Fine Tuned output']) == 'Irrelevant':
            df.at[i, 'labels_to_numbers (Not Fine Tuned)'] = 3
            
    for i in range(df.shape[0]):
        if(df.at[i, 'Llama3.1 Fine Tuned output']) == 'Favor':
            df.at[i, 'labels_to_numbers (Fine Tuned)'] = 0
        elif(df.at[i, 'Llama3.1 Fine Tuned output']) == 'Against':
            df.at[i, 'labels_to_numbers (Fine Tuned)'] = 1
        elif(df.at[i, 'Llama3.1 Fine Tuned output']) == 'Neutral':
            df.at[i, 'labels_to_numbers (Fine Tuned)'] = 2
        elif(df.at[i, 'Llama3.1 Fine Tuned output']) == 'Irrelevant':
            df.at[i, 'labels_to_numbers (Fine Tuned)'] = 3
    df.dropna(inplace=True)

    

    true = df['labels_to_numbers_original'].astype('int').to_numpy()
    pred = df['labels_to_numbers (Fine Tuned)'].astype('int').to_numpy()

    acc = accuracy_score(true, pred)
    target_names = ['favor', 'irrelavant', 'neutral', 'against']
    print(classification_report(true, pred, target_names=target_names))
    print("Accuracy with fine tuning: ", acc)
    
    cm = confusion_matrix(true, pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                  display_labels=['favor', 'irrelavant', 'neutral', 'against'])
    disp.plot()
    plt.show()
metric_display(df)

In [ ]:
text = '''

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are provided with an input. You are required to perform stance detection on the input with output as one of the following labels - Favor, Against, Irrelevant, Neutral. The labels are self-explanatory. Remember to only use the labels provided and do not mispell its name. Only output the stance detected label and nothing else.

### Input:
The Real OG of all times

### Response:
Favor<|eot_id|>

'''

match = re.search(r'### Response:\s*(\w+)', text)
match.group(1) if match else None


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 8,
    lora_dropout = 0.01, #
    bias = "none",    
   
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,  
    loftq_config = None, 
)

In [ ]:

df = pd.read_excel("/kaggle/input/frankdataset/Frank Lampart Ghost Goal llama 3.1 8b annotated (15).xlsx")
df = df[['Comments', 'Label']]
dataset = Dataset.from_pandas(df)

In [ ]:


system = ("You are provided with an input. You are required to perform stance detection "
              "on the input with output as one of the following labels - Favor, Against, "
              "Irrelevant, Neutral. The labels are self-explanatory. Only output the stance detected label.")

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


texts = []

EOS_TOKEN = tokenizer.eos_token



def formatting_function(item):



    system = ("You are provided with an input. You are required to perform stance detection "
              "on the input with output as one of the following labels - Favor, Against, "
              "Irrelevant, Neutral. The labels are self-explanatory. Only output the stance detected label.")


    user = item["Comments"]
    assistant = item["Label"]

    texts = []
    labels = []
    for input_text, output_text in zip(user, assistant):
        text = alpaca_prompt.format(system, input_text, output_text) + tokenizer.eos_token
        texts.append(text)
        labels.append(output_text)

    return {"text": texts, "labels": labels}


formatted_dataset = dataset.map(formatting_function, batched=True)

def tokenize_function(examples):
 
    tokenized_inputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=1024)

    tokenized_labels = []
    labels = []

        
    tokenized_labels = tokenizer(examples['labels'], padding="max_length", max_length=20)["input_ids"]


    return tokenized_inputs




In [ ]:
from datasets import Dataset

train_dataset, val_dataset = formatted_dataset.train_test_split(test_size=0.2, seed=42).values()
val_dataset, test_dataset = val_dataset.train_test_split(test_size=0.1, seed=42).values()

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,

    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1,
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "paged_adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        do_eval=True,
        evaluation_strategy="steps",  
        eval_steps=50, 
        report_to = "none",
        overwrite_output_dir = True
    ),
)

In [ ]:

gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:

used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

In [ ]:
# FastLanguageModel.for_inference(model)
# df = evaluate(model, 'Llama3.1 Fine Tuned output')
# df
# # metric_display(df)

In [ ]:
# model.save_pretrained("Llama3.1_8b_Instruct_lora_model")  # Local saving
# tokenizer.save_pretrained("Llama3.1_8b_Instruct_lora_model")
model.push_to_hub("YuvrajSingh9886/Llama3.1_8b_Instruct_lora_model_Lampard", token = "...") # Online saving
tokenizer.push_to_hub("YuvrajSingh9886/Llama3.1_8b_Instruct_lora_model_Lampard", token = "...") # Online saving